In [29]:
def logParse(parse):
    diclogs = {}
    u = open("userlog.log")
    file = u.readlines()
    u.close()
    
    # adding data to date_log {date:[{user:{session}, ....]}
    for line in file:
        column = line.split()
        date = column[0]
        if date in diclogs:
            if column[4] in diclogs[date]: # if user is already in the date append sesion info
                diclogs[date][column[4]].append({"time": column[1], "activity": column[2], "server": column[3]})
            else: # if date exists but user does not, create user
                diclogs[date][column[4]] = [{"time": column[1], "activity": column[2], "server": column[3]}]
        else: # add date and user
            diclogs[date] = {column[4]: [{"time": column[1], "activity": column[2], "server": column[3]}]}
    
    # sorting by date
    diclogs_copy = diclogs
    diclogs = {}
    for date in sorted(diclogs_copy):
        diclogs[date] = diclogs_copy[date]
        
    # sorting each member sessions by time
    for date in diclogs:
        for user in diclogs[date]:
            diclogs[date][user] = sorted(diclogs[date][user], key=lambda k: k['time']) 
    
    return diclogs

#suspicious
def suspicious(diclogs):
    susdic ={}
    login = 0
  
    for date in diclogs:
        susdic[date]={}
        for person in diclogs[date]:
            login = 0
            for time in diclogs[date][person]:
                if time['activity'] =='login':
                    login +=1
                    if login == 5:
                        susdic[date][person] = diclogs[date][person]

                        break
                    elif int(time['time'][:2]) >=5 and int(time['time'][:2])<= 12:
                        susdic[date][person] = diclogs[date][person]

                        break

    return susdic    

def irresponsible(diclogs):
    irrdic= {}
    login = 0
    logout = 0
    for date in diclogs:
        irrdic[date]={}
        for l in diclogs[date]:
            login = 0
            logout = 0
            for p in diclogs[date][l]:
                if p['activity'] == 'login':
                    login +=1
                else:
                    logout +=1
            if login>logout:

                irrdic[date][l]= diclogs[date][l]
    return irrdic


def glitches_in_system(diclogs):
    number_of_logins = 0
    number_of_logouts = 0
    word = {}
    for period in diclogs:
        word[period] = {}
        for person in diclogs[period]:
            number_of_logins = 0
            number_of_logouts = 0
            for time in diclogs[period][person]:
                if time['activity'] == 'login':
                    number_of_logins += 1
                else:
                    number_of_logouts += 1
            if number_of_logins < number_of_logouts:
                word[period][person] = diclogs[period][person]
    return word

def counting_the_domains(file):
    u = open(file, "r")
    words = u.readlines()
    getting_the_domains = {}
    u.close()
    for s in words:
        divide = s.split()
        person = divide[4].split('@')[0]
        domain = divide[4].split('@')[1]
        if domain in getting_the_domains:
            getting_the_domains[domain].add(person)
        else:
            getting_the_domains[domain] = set()
            getting_the_domains[domain].add(person)
    u.close()
    return getting_the_domains

def file_system_report(file_name, info, action):
    # refresh if making a new report with same name
    u = open(file_name, "w")
    u.write("")
    u.close()
    u = open(file_name, "a")
    result = 0
    addMode = {}
    number = {}
    for interval in info:
        result += len(info[interval])
    u.write("==============================\n" + "=== " + str(action) + "  (" + str(result) + " cases)" + " ===" + "\n==============================\n")

    count = 0
    for interval in info:
        for person in info[interval]:
            if person in number:
                number[person] = number[person] + 1
            else:
                number[person] = 1
            if person in addMode:
                addMode[person].append(f"\tDATE: [{interval}]---\n")
                for time in info[interval][person]:
                    addMode[person].append(f"\t\t\t{time['time']}\t\t{time['activity']}\t\t{time['server']}\n")
            else:
                addMode[person] = []
                addMode[person].append(f"\tDATE: [{interval}]---\n")
                for time in info[interval][person]:
                    addMode[person].append(f"\t\t\t{time['time']}\t\t{time['activity']}\t\t{time['server']}\n")
    for user in addMode:
        u.write(f"{person}\t\t\t{number[person]}\n")
        u.write("".join(addMode[person]))
    u.close()

def file_domain_in_system(file_name, domains):
    u = open(file_name, "w")
    u.write("")
    u.close()
    u = open(file_name, "a")
    u.write("==============================\n" + "=== DOMAIN COUNT " + "(" + str(len(domains)) + " DOMAINS" + ")"+ " ===" + "\n==============================\n")

    for domain in domains:
        u.write(f"{domain.rjust(20)}{str(len(domains[domain])).rjust(12)}\n") 
     
diclogs = logParse("userlog.log")

In [30]:
file_system_report("irresponsible_behavior_in_system.txt", irresponsible(diclogs), "IRRESPONSIBLE BEHAVIOR")
file_system_report("suspicious_activities_in_system.txt", suspicious(diclogs), "SUSPICIOUS ACTIVITY")
file_system_report("glitches_in_the_system_report.txt", glitches_in_system(diclogs), "SYSTEM GLITCH")
file_domain_in_system("user_domains_in_system_report.txt", counting_the_domains("userlog.log"))